In [1]:
%%writefile src/SimpleChild.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract Parent {
    //state variables
    address owner; //as of 0.8.0 no need to be address payable owner;
    uint private counter;
    
    //constructor. no need for constructor to be public as of 0.7.0
    constructor() {
        owner = msg.sender;
        counter = 0;
    }
    //functions
    function add() public { counter++; }
    function getCounter() public view returns(uint) { return counter; }
}

contract SimpleChild is Parent {
    //state variables
    string nickName;
    mapping(address => uint) private balances;
    //event
    event PrintLog(address, uint);
    
    //constructor
    constructor() {}  // no public as in constructor() public {}  
    //functions
    function setNickName(string memory s) public { nickName = s; }
    function getNickName() public view returns(string memory) { return nickName; }
    function deposit() public payable {
        balances[msg.sender] += msg.value;
        emit PrintLog(msg.sender, msg.value);
    }
    function queryBalance() public view returns (uint) {
        return balances[msg.sender];
    }
    //access non-private members of the parent
    function kill() public {
        if (msg.sender == owner) selfdestruct(payable(owner)); //0.6.x selfdestruct(owner)
    }
}

Overwriting src/SimpleChild.sol


In [2]:
!solc-windows.exe src/SimpleChild.sol

Compiler run successful, no output requested.


In [3]:
%%writefile src/IntBool.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract IntBoolTest {
    bool married = true;
    uint256 xAge = 22;
    uint256 yAge = 25;
    //fixed phi; // = 3.14;
    function update() public {
        xAge = yAge;
        yAge = 33;
    }
    function setXAge(int age) public {
        xAge = uint(age);  //type conversion
    }
    function getXAge() public view returns(uint) {
        return xAge;
    }
    function getYAge() public view returns(uint) {
        return yAge;
    }
    function testInt() public view returns(bool) {
        assert(xAge>=20 && yAge>=20);
        return true;
    }
    function isMarried() public view returns(bool) {
        return married;
    }
}

Overwriting src/IntBool.sol


In [4]:
!solc-windows.exe src/IntBool.sol

Compiler run successful, no output requested.


In [5]:
%%writefile src/ByteStringTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract ByteStringTest {
    //byte b = 0xFF; //use bytes1 as of 0.8.0
    bytes1 b1 = 0xFF;
    bytes2 b2 = 0xFFAA;
    bytes8 place8 = "7 hongji"; //8 characters long
    bytes23 place23 = "7 hongji-dong jongro-gu";
    bytes place = "7 hongji-dong jongro-gu Seoul"; //variable length
    bytes myBytes = new bytes(3);  //0x000000
    string constant name = "jsl"; //utf-8 string "jsl"
    function getB1() public view returns(bytes1) {
        return b1;  //bytes1, so no casting required
    }
    function getB2() public view returns(bytes2) {
        return b2;
    }
    function getB23() public view returns(bytes23) {
        return place23;  //fixed size, value type (no memory)
    }
    /**@return hex bytes. reference type should be set as memory*/
    function getBytes() public view returns(bytes memory) {
        return myBytes;  //smu in hex 0x736d75
    }
    function getLengOfBytes23 () view public returns(uint) {
        return place23.length;  // returns 23
    }
    function getLenOfBytes() pure public returns(uint) {
        bytes memory bm = "7 hongji-dong jongro-gu";
        return bm.length;        // returns 23
    }
    //need the arg in hex e.g. bytes1 0x61 bytes2 0x6161
    //a 61, b 62, ... , y 79
    //try invalid type, e.g. bytes2 0x61 or 0x616161
    function setB2(bytes2 _b2) public {
        b2 = _b2;
    }
    function setBytes() public {
        myBytes = "smu";
    }
    function getLenOfString() pure public returns(uint) {
        string memory nameLocal = "jslLocal";
        //return nameLocal.length;  //error, casting required
        return bytes(nameLocal).length;
    }
    function getString() pure public returns(string memory) {
    //function getString() pure public returns(bytes memory) {
        string memory s = "\xec\x95\x88\xeb\x85\x95"; //"한글";
        //bytes memory s = "\xec\x95\x88\xeb\x85\x95"; //"한글";
        return s;
    }
}

Overwriting src/ByteStringTest.sol


In [6]:
!solc-windows.exe src/IntBool.sol

Compiler run successful, no output requested.


In [7]:
%%writefile src/StructTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract StructTest {
    struct Student {
        uint num;
        string name;
        bool isEnrolled;
    }
    Student s1=Student(201911111,"jslim",true);
    Student s2;
    //memory only for string type
    //201711111,"kim",false
    function setStudent2(uint n, string memory sn, bool e) public {
        s2.num = n;
        s2.name = sn;
        s2.isEnrolled = e;
    }
   function getStudent1() public view returns(uint, string memory, bool){
       return (s1.num, s1.name, s1.isEnrolled);
   }
   function getStudent2() public view returns(uint, string memory, bool){
       return (s2.num, s2.name, s2.isEnrolled);
   }
   function getStudentName() pure public returns(string memory) {
       //the right is locally created, so memory (not storage) is declared
       Student memory s3 = Student(201911112, "jsl3", true);
       return s3.name;
   }
}

Overwriting src/StructTest.sol


In [8]:
!solc-windows.exe --bin src/StructTest.sol


======= src/StructTest.sol:StructTest =======
Binary:
60806040526040518060600160405280630c08eb4781526020016040518060400160405280600581526020017f6a736c696d000000000000000000000000000000000000000000000000000000815250815260200160011515815250600080820151816000015560208201518160010190805190602001906100829291906100b7565b5060408201518160020160006101000a81548160ff02191690831515021790555050503480156100b157600080fd5b506101bb565b8280546100c39061015a565b90600052602060002090601f0160209004810192826100e5576000855561012c565b82601f106100fe57805160ff191683800117855561012c565b8280016001018555821561012c579182015b8281111561012b578251825591602001919060010190610110565b5b509050610139919061013d565b5090565b5b8082111561015657600081600090555060010161013e565b5090565b6000600282049050600182168061017257607f821691505b602082108114156101865761018561018c565b5b50919050565b7f4e487b7100000000000000000000000000000000000000000000000000000000600052602260045260246000fd5b61071b806101ca6000396000f3fe60806040523480156100105760008

In [9]:
!solc --abi --bin --gas src/StructTest.sol


======= src/StructTest.sol:StructTest =======
Gas estimation:
construction:
   infinite + 363800 = infinite
external:
   getStudent1():	infinite
   getStudent2():	infinite
   getStudentName():	infinite
   setStudent2(uint256,string,bool):	infinite
Binary:
60806040526040518060600160405280630c08eb4781526020016040518060400160405280600581526020017f6a736c696d000000000000000000000000000000000000000000000000000000815250815260200160011515815250600080820151816000015560208201518160010190805190602001906100829291906100b7565b5060408201518160020160006101000a81548160ff02191690831515021790555050503480156100b157600080fd5b506101bb565b8280546100c39061015a565b90600052602060002090601f0160209004810192826100e5576000855561012c565b82601f106100fe57805160ff191683800117855561012c565b8280016001018555821561012c579182015b8281111561012b578251825591602001919060010190610110565b5b509050610139919061013d565b5090565b5b8082111561015657600081600090555060010161013e565b5090565b6000600282049050600182168061017257607f821691505b6

In [10]:
%%writefile src/EnumTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract EnumTest {
    enum Day {MONDAY,TUESDAY,WEDNESDAY,THURSDAY,FRIDAY,SATURDAY,SUNDAY}
    Day myDay = Day.FRIDAY; //index int4
    
    /* @return Day  returning index*/
    function getMyDay() public view returns(Day) {
        return myDay;   //index
    }
    /* @param d  ok to pass an integer (uint8)*/
    function setMyDay(Day d) public {
        myDay = d;
    }
    //uint is converted to uint8, which is default
    function setMyDayInt(uint d) public {
        myDay = Day(d);
    }
}

Overwriting src/EnumTest.sol


In [11]:
%%writefile src/ArrayTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract ArrayTest {
    uint[3] ages = [15, 25, 35];
    int[] marks; //dynamic ArrayTest
    
    /* @param index  array index
       @param val    value at the index*/
    function updateAges(uint index, uint val) public {
        if(index>=0 && index <=2)
            ages[index] = val;
    }
    function initMarks() public {
        marks = new int[](5);   // default 0
    }
    function appendMark(int mark) public {
        marks.push(mark);
    }
    function popMark() public {
        marks.pop();
    }
    /* @return dynamic array, so memory is used*/
    function getMarks() public view returns(int[] memory) {
        return marks;
    }
    /* @return fixed array, so memory is used*/
    function getAges() public view returns(uint[3] memory) {
        return ages;
    }
    function getLenOfArr() pure public returns(uint) {
        //memory is used because locally created array is assigned
        uint8[3] memory intArr = [0, 1, 2];
        return intArr.length;
    }
}

Overwriting src/ArrayTest.sol


송금시 3가지 단어를 모두 기억해야함 send(), transfer(), call.value()
### send, transfer, call.value 순으로
### gas 수정가능 : No, No, Yes
### gas 한도 : 2300, 2300, 모든가용 gas를 넘겨주거나 일정gas를 정할 수 있음
### 실패시(송금실패) : false, throw 예외 발생, false

#### transfer
- gas 수정불가능, gas한도 2300, 송금실패시 throw 예외 발생
- transfer 함수로 송금시 수신자 측에서 receive, fallback함수를 통해 수신됨
- 수신자주소.transfer(금액) --> 수신자주소로 금액만큼 송금

#### send
- gas 수정불가능, gas한도 2300, 송금실패시 false 반환
- send 함수로 송금시 수신자 측에서 receive, fallback함수를 통해 수신됨
- require(수신자주소.send(금액)) --> 수신자주소로 금액만큼 송금

#### call
- gas 수정가능, gas한도 수정가능, 송금 실패시 false 반환
- transfer와 send는 수신자측에 gas부담을 시키는 공격이 가능하여 call이 더 안전하다는 주장도 있음
- (bool success, ) = <address>.call{value:20}(""); //20 Wei를 송금한다. 괄호가 없으면 값을 설정했다는 의미, 소괄호 안에 비어있으므로 수신함수가 없음
- (bool success, ) = <address>.call{gas:2, value:11111}(""); //처럼 가스를 명시할 수 도 있음

In [13]:
%%writefile src/AddressTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract AddressTest {
    address owner;
    address payable receiver; //address receiver;
    uint balanceOfOwner;
    constructor() { // NO! constructor() public {
        owner=msg.sender;
        //myBalance = msg.sender.balance;
        balanceOfOwner = owner.balance;
    }
    function deposit() payable public {
    }
    /* @param addr  set as payable because it will get some gwei*/
    function setReceiver(address payable addr) public {
        receiver=addr;
    }
    function getReceiver() view public returns(address) {
        return receiver;
    }
    function getBalanceOfThis() public view returns(uint) {
        return address(this).balance;  //balance of contract
    }
    function getBalanceOfOwner() public view returns(uint) {
        return owner.balance;
    }
    function getBalanceOfReceiver() public view returns(uint) {
        return receiver.balance;
    }
    function send() public payable {
        require(receiver.send(111)); //send 111 gwei to xAddress
    }
    function transfer() public payable {
        //if !(receiver.transfer(address(this).balance))
        receiver.transfer(11111);
    }
    function callValue() public payable {
        //receiver.call.value(11111)(""); //deprecated
        //receiver.call{value: 11111}("");  //Warning: Return value of low-level calls not used.
        //(bool success, bytes memory data) = receiver.call{value: 11111}(""); //waring persists
        (bool success, ) = receiver.call{value: 11111}("");
        require(success, "transfer call failed.");
        //receiver.call.gas(10).value(11111)("");  //warning
        //receiver.call{gas: 10, value: 11111}(""); //warning
        (success, ) = receiver.call{gas: 10, value: 11111}("");
        require(success, "transfer call failed.");
    }
}
    

Overwriting src/AddressTest.sol


### Mybank.sol 실행시

In [14]:
%%writefile src/MyBank.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract MyBank {
    address owner; //address payable owner;
    uint balance;
    constructor() { //constructor() public {
        owner = msg.sender;
        balance = address(this).balance;
    }
    function deposit(uint amount) public payable {
        require(msg.value == amount);
        balance += amount;
    }
    function withdraw(uint amount) public payable {
        balance -= amount;   // deduct before transfer
        payable(owner).transfer(amount); //owner.transfer(amount);
    }
    function transferTo(address payable receiver, uint amount) public payable {
        balance -= amount;   // deduct before transfer
        receiver.transfer(amount);
    }
    function getBalance() public view returns (uint) {
        return balance;
    }
    function getBalanceOfThis() public view returns (uint) {
        return address(this).balance;
    }
    function getBalanceOfOwner() public view returns (uint) {
        return owner.balance;
    }
}

Overwriting src/MyBank.sol


In [17]:
%%writefile src/FuctionTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract FunctionTest {
    int x;
    //constructor
    constructor() { //constructor() public {
        x = 0;
    }
    // updating state var x
    function incrementX() public {
        x += 1;
    }
    // call when x = 0
    function doubleX() public {
        X2();
    }
    // float not supported. try 0, 1/3...
    function divideBy(int by) view public returns(int) {
        return x/by;
    }
    // actually NONE returned
    function getX_() view public returns(int) {
        return x;        
    }
    // try pure
    function getX() view public returns(int) {
        return x;
    }
    function getBalance() view public returns(uint) {
        return(address(this).balance);
    }
    // 'payable' means that msg.value can be deposited
    function deposit() public payable {
    }
    // can not be used in public
    function X2() internal {
        x *= 2;
    }
    function getBlockNumber() view public returns(uint) {
        return block.number;
    }
}

Overwriting src/FuctionTest.sol


In [18]:
%%writefile src/UnderscoreTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract UnderscoreTest {
    string season = "";
    function getSeason() view public returns(string memory) {
        return season;
    }
    function setWinter() public setSummerAfter {
        season = "winter";
    }
    function setSpring() public setSummerBefore {
        season = "spring";
    }
    modifier setSummerAfter() {
        season = "summer";
        _;
    }
    modifier setSummerBefore() {
        _;
        season = "summer";
    }
}

Overwriting src/UnderscoreTest.sol


In [19]:
%%writefile src/webSocketTest.js
var Web3 = require('web3');
const myProvider = new Web3.providers.WebsocketProvider("ws://localhost:8345", {
    clientConfig: {
        keepalive:true, keepaliveInterval:10000
    } 
  });
var web3 = new Web3(myProvider);
console.log("(1) websocket url: ", myProvider.connection.url); //web3.currentProvider.connection.url
myProvider.on('connect', function() {
    console.log("(2) connecting websocket: "+web3.currentProvider.connected);
    //myProvider.disconnect();
    web3.currentProvider.connection.close();
    console.log("(3) disconnecting Websocket: "+web3.currentProvider.connected);
});
myProvider.on('close', function() { console.log("--> Websocket closed"); });
myProvider.on('end', function() { console.log("--> Websocket ended"); });
myProvider.on('error', function(error) { console.error(error); });

Writing src/webSocketTest.js


In [20]:
!node src/webSocketTest.js

(1) websocket url:  ws://localhost:8345
(2) connecting websocket: true
(3) disconnecting Websocket: false
--> Websocket ended


# Event 이벤트 발생

In [35]:
%%writefile src/EventTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract EventTest {
    event MyLog(string my);
    function myFunction() public {
        emit MyLog("Hello World!");
    }
}

Overwriting src/EventTest.sol


In [11]:
!solc-windows.exe --optimize --combined-json abi,bin src/EventTest.sol > src/EventTest.json

!solc-windows.exe --optimize --combined-json abi,bin src/EventTest.sol > src/EventTest.json 은 0.8.0의 exe이름을 solc-windows.exe로 하였을때의 경우

In [143]:
%%writefile src/EventTestDeployFromFile.js
var Web3 = require('web3');
var _abiBinJson = require('./EventTest.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts); // reading ['src/EventTest.sol:EventTest']
console.log("- contract name: ", contractName);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!! //SyntaxError: Unexpected token o in JSON at position 1
_bin=_abiBinJson.contracts[contractName].bin;

console.log("- ABI: " + JSON.stringify(_abi));
console.log("- Bytecode: " + _bin);

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 259210}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/EventTestDeployFromFile.js


In [144]:
!node src/EventTestDeployFromFile.js

- contract name:  [ 'src/EventTest.sol:EventTest' ]
- ABI: [{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"my","type":"string"}],"name":"MyLog","type":"event"},{"inputs":[],"name":"myFunction","outputs":[],"stateMutability":"nonpayable","type":"function"}]
- Bytecode: 6080604052348015600f57600080fd5b5060c68061001e6000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c8063c3780a3a14602d575b600080fd5b60336035565b005b7f5186edd9beca61d795526ca1f274260b3fc74be3e10e1f02e1be1552e14f1373604051606090606a565b60405180910390a1565b6020808252600c908201526b48656c6c6f20576f726c642160a01b60408201526060019056fea2646970667358221220dfbd40787b99f786bcf3d200eba556a0b7bf3364be90dae7e4aca9c614c9583964736f6c63430008000033
Deploying the contract from 0xB52ed7C04e5F6E1aD9f381b3e8CC6Eb1D867e6fD
hash: 0x59c03517cc3759b1ced635637a360952e1d15276a2f76bdf047b397571fba076
---> The contract deployed to: 0xC7C680Df4AAe44430B85bda2B3509315CF3E46AE


In [145]:
%%writefile src/EventTestHttpNoEventFiredUse.js
var Web3=require('web3');
var _abiBinJson = require('./EventTest.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
contractName=Object.keys(_abiBinJson.contracts); // reading ['src/EventTest.sol:EventTest']
//console.log("- contract name: ", contractName); //or console.log(contractName[0]);
_abiArray=_abiBinJson.contracts[contractName].abi; //use just as read from file
//_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!!
//_bin=_abiBinJson.contracts[contractName].bin;
console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);
var _test = new web3.eth.Contract(_abiArray,"0xC7C680Df4AAe44430B85bda2B3509315CF3E46AE");
var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
    if (!error) {
        console.log("Event fired: " + JSON.stringify(result.returnValues));
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    const value = await _test.methods.myFunction()
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));});
    console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}
doIt()

Overwriting src/EventTestHttpNoEventFiredUse.js


In [146]:
!node src/EventTestHttpNoEventFiredUse.js

- ABI: [object Object],[object Object]
Account: 0xB52ed7C04e5F6E1aD9f381b3e8CC6Eb1D867e6fD
Balance before: 999996230379000000000
---> myFunction called {"0":"Hello World!","my":"Hello World!"}
Balance after: 999996207552000000000
Balance diff: 22827000004608


In [159]:
%%writefile src/EventTestWsUse.js
var Web3=require('web3');
var fs = require('fs');
var _abiBinJson = require('./EventTest.json');      //importing a javascript file

//var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
//var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345"));
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
contractName=Object.keys(_abiBinJson.contracts); // reading ['src/EventTest.sol:EventTest']
//console.log("- contract name: ", contractName); //or console.log(contractName[0]);
_abiArray=_abiBinJson.contracts[contractName].abi; //use just as read from file
//_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!!
//_bin=_abiBinJson.contracts[contractName].bin;
console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);
//var _test = new web3.eth.Contract(_abiArray,"0xC7C680Df4AAe44430B85bda2B3509315CF3E46AE");
//var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
//    if (!error) {
//        console.log("Event fired: " + JSON.stringify(result.returnValues));
//    }
//});

async function doIt() {
    var _test = new web3.eth.Contract(_abiArray, '0xC7C680Df4AAe44430B85bda2B3509315CF3E46AE');
    var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
        if (!error) {
            log = JSON.stringify(result.returnValues);
            console.log("Event fired: " + log);
            //fs.writeFile("src/EventTestLog.txt", log, "utf-8", function(e) {
            fs.appendFile("src/EventTestLog.txt", log, "utf-8", function(e) {
                if(!e) {
                    console.log(">> Writing to file");
                }
            });
        }
    });
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    const value = await _test.methods.myFunction()
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));});
    console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    process.exit(1); //force exit to disconnect websocket
}

doIt()

Overwriting src/EventTestWsUse.js


In [160]:
!node src/EventTestWsUse.js

- ABI: [object Object],[object Object]
Event fired: {"0":"Hello World!","my":"Hello World!"}
Account: 0xB52ed7C04e5F6E1aD9f381b3e8CC6Eb1D867e6fD
>> Writing to file
Balance before: 999996116244000000000
Event fired: {"0":"Hello World!","my":"Hello World!"}
>> Writing to file
---> myFunction called {"0":"Hello World!","my":"Hello World!"}
Balance after: 999996093417000000000
Balance diff: 22827000004608


# solc 두가지 버전도 사용가능 solc.exe (0.4.25), solc-windows.exe (0.8.0)

# fallback

In [161]:
%%writefile src/FallbackTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract FallbackTest {
    event PrintLog(string);
    function callA () pure public returns(string memory){
        return "doing callA";
    }
    fallback () external {
        emit PrintLog("fallback called");
    }
}

Writing src/FallbackTest.sol


In [200]:
!solc --optimize --combined-json abi,bin src/FallbackTest.sol > src/FallbackTest.json

In [201]:
%%writefile src/FallbackTestDeployFromFile.js
var Web3 = require('web3');
var _abiBinJson = require('./FallbackTest.json');      //importing a javascript file
//var fs=require('fs');
//var _str = fs.readFileSync("src/FallbackTest.json");
//var _json=JSON.parse(_str)

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts); // reading ['src/FallbackTest.sol:FallbackTest']
console.log("- contract name: ", contractName);
_abiArray=_abiBinJson.contracts[contractName].abi;
//_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!! //SyntaxError: Unexpected token o in JSON at position 1
_bin=_abiBinJson.contracts[contractName].bin;

//console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/FallbackTestDeployFromFile.js


In [202]:
!node src/FallbackTestDeployFromFile.js

- contract name:  [ 'src/FallbackTest.sol:FallbackTest' ]
Deploying the contract from 0xB52ed7C04e5F6E1aD9f381b3e8CC6Eb1D867e6fD
hash: 0x2ae6b8b6cb414eaf7b39f028286fde7916b9323c8a8af5c702152e62a7a852b3
---> The contract deployed to: 0x64A4Abe0A1F7ba61d4a9EC6191b651c4848ccb58


In [224]:
%%writefile src/FallbackTestUseFromFile.js
var Web3=require('web3');
var _abiBinJson = require('./FallbackTest.json');      //importing a javascript file

//var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
contractName=Object.keys(_abiBinJson.contracts); // reading ['src/FallbackTest.sol:FallbackTest']
console.log("- contract name: ", contractName); //or console.log(contractName[0]);
_abiArray=_abiBinJson.contracts[contractName].abi; //use just as read from file
//_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!!
//_bin=_abiBinJson.contracts[contractName].bin;
console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);
//The above ABI from file does not work. So ABI was copied and pasted as below -> It worked!
_abiArray=[{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"","type":"string"}],"name":"PrintLog","type":"event"},{"stateMutability":"nonpayable","type":"fallback"},{"inputs":[],"name":"callA","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"pure","type":"function"}];
async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    var _instance = new web3.eth.Contract(_abiArray, "0x64A4Abe0A1F7ba61d4a9EC6191b651c4848ccb58");
    var event = _instance.events.PrintLog({fromBlock: 0}, function (error, result) {
        if (!error) {
            console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
        }
    });

    _instance.methods.callA().call().then(function(res) { console.log(res); });  //null
    //call without calling any method
    //await _instance.methods.callB().send({from:accounts[0], to: "0x3991e87b71cBFf94aA0718F341d8Ad4bCF969f36"}); //fail
    //await _instance.methods.callA().send({from:accounts[0], data:"0x1234"});  //empty calldata to call fallback -> fail
    web3.eth.sendTransaction({from:accounts[0], to:"0x64A4Abe0A1F7ba61d4a9EC6191b651c4848ccb58"}); //fallback called
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    process.exit(1); //force exit
}
doIt()

Overwriting src/FallbackTestUseFromFile.js


In [226]:
!node src/FallbackTestUseFromFile.js

- contract name:  [ 'src/FallbackTest.sol:FallbackTest' ]
- ABI: [object Object],[object Object],[object Object]
Account: 0xB52ed7C04e5F6E1aD9f381b3e8CC6Eb1D867e6fD
Balance before: 999994322995000000000
Event fired: {"address":"0x64A4Abe0A1F7ba61d4a9EC6191b651c4848ccb58","blockHash":"0xcfdf05a5d89c961d37911c8a4f93fda976f0629422573cf589a35b91361bcc20","blockNumber":38,"logIndex":0,"removed":false,"transactionHash":"0x66196e597ace3590ac41351af2486d013c26e0de5dce01ce7b530283f787fce5","transactionIndex":0,"id":"log_b51f1197","returnValues":{"0":"fallback called"},"event":"PrintLog","signature":"0x968f0302429ff0e5bd56a45ce3ba1f4fa79f4b822857e438616435f00c3b59fd","raw":{"data":"0x0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000f66616c6c6261636b2063616c6c65640000000000000000000000000000000000","topics":["0x968f0302429ff0e5bd56a45ce3ba1f4fa79f4b822857e438616435f00c3b59fd"]}}
---> {"0":"fallback called"}
Event fired:

# 반복문
## LoopTest sum을 local(Memory)에 두는 경우와 state variable(Storage)에 두는 경우 비교

In [227]:
%%writefile src/LoopTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract LoopTest {
    uint sum;
    constructor() { //constructor() public {
        init();
    }
    function forLoop() public {
        //init();
        uint sumLocal = 0;
        for(uint i = 0; i < 101; i++) {
            sumLocal += i;
        }
        sum = sumLocal;
    }
    function forLoopCostly() public {
        init();
        //uint sumLocal = 0;
        for(uint i = 0; i < 101; i++) {
            //sumLocal += i;
            sum += i;
        }
        //sum = sumLocal;
    }
    function whileLoop() public {
        uint i = 0;
        uint sumLocal = 0;
        while(i <= 100) {
            sumLocal += i;
            i++;
        }
        sum = sumLocal;
    }
    function whileLoopBreak() public {
        uint sumLocal = 0;
        uint i = 0;
        while(true) {
            sumLocal += i;
            if(i==100) break;
            i++;
        }
        sum = sumLocal;
    }   
    function doWhileLoop() public {
        uint i = 0;
        uint sumLocal = 0;
        do {
            sumLocal += i;
            i++;
        } while(i <= 100);
        sum = sumLocal;
    }
    function init() public { sum = 0; }
    function getSum() view public returns(uint) { return sum; }
}

Writing src/LoopTest.sol


리믹스에서 실행시
1. forloop는 local에서 업데이트  --> 94192, forloopcostly는 state variable --> 99911
2. while의 경우 break가 있는것이 더 많음

# 조건문

In [228]:
%%writefile src/ConditionalTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract ConditionalTest {
    function toLowerCase(string memory str) pure public returns (string memory) {
        bytes memory bIn = bytes(str); //str은 자동으로 memory에 들어가기때문에 bIn의 자료형은 bytes memory
        bytes memory bOut = new bytes(bIn.length);
        uint8 _iIn = 0;
        for (uint i = 0; i < bIn.length; i++) {
            _iIn = uint8(bIn[i]);
            if (_iIn >= 65 && _iIn <= 90)    // ascii A:65 ~ Z:90                
                bOut[i] = bytes1(_iIn + 32); // ascii a:97 ~ z:122
            else 
                bOut[i] = bIn[i];
        }
        return string(bOut);
    }
    function abs(int i) pure public returns (int) {
        return (i > 0) ? i : -i;
    }
}

Writing src/ConditionalTest.sol


# Hello1

In [229]:
%%writefile src/Hello1.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract Hello1 {
    string private hello;
    address private owner;
    event PrintLog(address addr, string s);

    constructor(string memory _hello) { //constructor(string memory _hello) public {
        hello = _hello;
        owner = msg.sender;
    }
    function sayHello() view public returns(string memory) {
        return hello;
    }
    modifier isOwner() {
        if (msg.sender != owner) {
            revert();
        }
        _; //insert here the calling function
    }
    function setHello() public {
        string memory s = "";
        if (msg.sender == owner) {
            s = "Hello";
        } else {
            s = "Olleh";
        }
        emit PrintLog(msg.sender, s);
        hello = s;
    }
    function compareTo(string memory _str) view public returns(bool) {
        bool isEqual = false;
        //if (hello == _str) {
        if (keccak256(bytes(hello)) == keccak256(bytes(_str))) {
            isEqual = true;
        }
        return isEqual;
    }
    function getBalance() view public isOwner returns(uint) {
        return address(this).balance;
    }
}

Writing src/Hello1.sol


## 랜덤함수 사용

In [3]:
%%writefile src/Random.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract Random {
    function rand() public view returns(bytes32) {
        //seed값을 설정
        return keccak256(abi.encodePacked(block.timestamp, block.difficulty));
    }
    function rand0and250() public view returns(uint8) {
        return uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty)))%251);
    }
    function rand0and9() public view returns(uint8) {
        return uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty)))%10);
    }
    function rand0and2() public view returns(uint8) {
        return uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty)))%3);
    }
    function genRandomInteger() view public returns(uint8[] memory) {
        uint8[] memory r = new uint8[](10);
        for(uint i = 0; i<r.length; i++)
            //r[i] = uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty)))%10);
            //change timestamp by adding i, otherwise all the same numbers generated
            r[i] = uint8(uint256(keccak256(abi.encodePacked(block.timestamp + i, block.difficulty)))%10);
            //seed를 임의로 바꿈으로써 항상 rand가 다르게 나오게끔 설정
        return r;
    }
}

Overwriting src/Random.sol


# 고정배열, 동적배열 (storage, memory 차이)

In [4]:
%%writefile src/ArrayTest2.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;
pragma experimental ABIEncoderV2;

contract ArrayTest2 {
    string[3] cities1=["Seoul", "Sydney", "Tokyo"];
    string[] cities2 = new string[](2);
    string[] cities3;
    uint24[5] balance = [255, 65536, 95, 50, 1];
    int[5] mathMarks;
    uint[3][] marks=[[100, 80, 95],[20,30,40]];

    function setLocalDynamicArr() pure public returns(uint) {
        //uint[] myArr; //error
        //uint[] storage myArr; //error uninitialized storage
        //uint[] memory myArr; //ok. but cannot push
        //uint[] myArr = new uint[](3); //error
        //uint[] storage myArr = new uint[](3); //error conversion from memory to stoarge
        uint[] memory myArr = new uint[](3);
        //myArr.push(11); //error
        myArr[0]=11;
        myArr[1]=12;
        //myArr.push(13); //error
        //myArr[5]=15;
        return myArr.length;
    }
    function getDynamicArrMemory() pure public returns(uint[] memory) {
        uint[] memory num=new uint[](3);  //dynamic
        for (uint i=0; i<num.length; i++)
            num[i]=i;       //push() not allowed for array memeory
        return num;
    }
    //string is a dynamic array itself, which can not be returned.
    //function setArrMemory() view public returns(string[] memory) {
    function getStringDynamicArrMemory() pure public {
        //array storage not allowed
        //error: string[2] storage places = ["9000", "Sydney"];
        string[2] memory places = ["9000", "Sydney"];
        //array memory push not allowed
        //places.push("Seoul");
        places[0]="Seoul";
        //return places;
    }
    /*returning 'string[] storage' is not allowed
    function getCities1_() view public returns(string[] memory) {
        return cities1;  //can not return stoarge var. 
    }*/
    function getCities1() view public returns(string memory) {
        return cities1[0];
    }
    function getCities1Length() view public returns(uint) { return cities1.length; }
    function setCities23() public {
        //string my = "Seoul"; //error
        string memory my = "seoul";
        cities2[0]="New York";
        cities2.push(my);
        cities2.push("Busan");
        cities3.push("New York");
        cities3.push("Beijing");
    }
    //dynamic array return needs "pragma experimental ABIEncoderV2;"
    function getCities2() view public returns(string[] memory){
        return cities2;
    }
    function setMathMarks() public {
        //mathMarks.push(100); //push for fixed array not allowed
        //mathMarks=uint8([100,60,95,50,80]); //error: uint8 memory -> uint8
        uint8[5] memory temp = [100, 60, 95, 50, 80];
        //mathMarks = int(temp); //error: uint8[5] memory -> int256
        for (uint i = 0; i < temp.length; i++) {
            //mathMarks[i] = int(temp[i]); //error: uint8 -> int256
            mathMarks[i] = int(int8(temp[i])); //ok: uint8 -> int8 -> int256
        }
    }
    function getMathMarks() view public returns (int[] memory) {
        //return mathMarks;  //can not return storage array: error: int[] storage -> int[] memory
        int[] memory _arr256 = new int[](mathMarks.length);
        for (uint i = 0; i < mathMarks.length; i++) {
            _arr256[i] = int256(mathMarks[i]);
        }
        return _arr256;
    }
    //run setMathMarks() beforehand
    function getMathAbove70_() view public returns(int[] memory) {
        // size is not allocated yet -> invalid opcode error
        int[] memory mathAbove70;
        uint counter = 0;
        for(uint8 i=0;i<mathMarks.length;i++)
            if(mathMarks[i]>70) {
                mathAbove70[counter] = mathMarks[i];
                //mathAbove70.push(mathMarks[i]);
                counter++;
            }
        return mathAbove70;
    }
    //run setMathMarks() beforehand
    function getMathAbove70() view public returns(int[] memory) {
        //compute lengthOfMathAbove70
        uint8 counter=0;
        uint8 lengthOfMathAbove70=0;
        for(uint8 i=0;i<mathMarks.length;i++)
            if(mathMarks[i]>70) counter++;
        lengthOfMathAbove70=counter;
        //move math marks above 70
        int[] memory mathAbove70=new int[](lengthOfMathAbove70);
        counter=0;
        for(uint i=0;i<mathMarks.length;i++) {
            if(mathMarks[i]>70) {
                mathAbove70[counter]=mathMarks[i];
                counter++;
            }
        }
        return mathAbove70;
    }
    function updateMarks() public returns(uint[3][] memory){
        marks[0][0]=90;
        return marks;
    }
    function getMarksArr() view public returns(uint[3][] memory) {
        return marks;
    }
    function getMarksLength() view public returns(uint) {
        return marks.length;
    }
}

Writing src/ArrayTest2.sol


In [31]:
!solc-windows.exe src/ArrayTest2.sol --combined-json abi > src/ArrayTest2ABI.json

In [32]:
!solc-windows.exe src/ArrayTest2.sol --combined-json bin > src/ArrayTest2BIN.json

In [51]:
%%writefile src/ArrayTest2Deploy.js
var Web3=require('web3');
var _abiJson = require('./ArrayTest2ABI.json');
var _binJson = require('./ArrayTest2BIN.json');

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
contractName=Object.keys(_abiJson.contracts); // reading ['src/ArrayTest2.sol:ArrayTest2']
console.log("- contract name: ", contractName[0]); //or console.log(contractName);
_abiArray=_abiJson.contracts[contractName].abi;
//_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);    //JSON parsing needed!!
_bin="0x"+_binJson.contracts[contractName].bin; //ok without "0x"

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);

    new web3.eth.Contract(_abiArray).deploy({data: _bin}).estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas);
    });

    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1621126}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address);
}

deploy()

Overwriting src/ArrayTest2Deploy.js


In [52]:
!node src/ArrayTest2Deploy.js

- contract name:  src/ArrayTest2.sol:ArrayTest2
Deploying the contract from 0x43D9120264e2340504fEEa14a2A59698dacfA66C
hash: 0xf39252a39ad5d5ae1f89b43a52a12ad08c5de392a68bb696ca811538d05e7144
---> The contract deployed to: 0xED2511d6167E31DB62C1A089D9413CAF6114FCA3
>> gas: 1621125


In [71]:
%%writefile src/ArrayTest2Use.js
var Web3=require('web3');
var _abiJson = require('./ArrayTest2ABI.json');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));       //ok

contractName=Object.keys(_abiJson.contracts); // reading ['src/ArrayTest2.sol:ArrayTest2']
console.log("- contract name: ", contractName[0]); //or console.log(contractName);
_abiArray=_abiJson.contracts[contractName].abi;
//_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);    //JSON parsing needed!!

async function doIt() {
    var arr = new web3.eth.Contract(_abiArray, "0xED2511d6167E31DB62C1A089D9413CAF6114FCA3");
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    
    arr.methods.setCities23().estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas);
    });

    //await arr.methods.setCities23().send({from: accounts[0]});   //out of gas error
    await arr.methods.setCities23().send({from: accounts[0], gas:1621125});
    arr.methods.getCities2().call().then(console.log);

    arr.methods.setMathMarks().estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas);
    });

    await arr.methods.setMathMarks().send({from: accounts[0], gas: 1621125});
    //ERROR invalid opcode arr.methods.getMathAbove70().call().then(console.log);
    arr.methods.getMarksArr().call().then(console.log);

    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()

Overwriting src/ArrayTest2Use.js


## 위에 use 시에 deploy에서 나온 gas비를 함수의 gas에 넣어서 실행해야 함

In [76]:
!node src/ArrayTest2Use.js

- contract name:  src/ArrayTest2.sol:ArrayTest2
Account: 0x43D9120264e2340504fEEa14a2A59698dacfA66C
Balance before: 999975589354000000000
>> gas: 127611
Balance after: 999975266512000000000
Balance diff: 322841999966208
[
  'New York', '',
  'seoul',    'Busan',
  'seoul',    'Busan',
  'seoul',    'Busan',
  'seoul',    'Busan',
  'seoul',    'Busan'
]
>> gas: 33810
[ [ '100', '80', '95' ], [ '20', '30', '40' ] ]


# MembersMap

## 10개의 주소에 각각 Member를 할당 (Id, Name)

In [79]:
%%writefile src/MembersMap.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract MembersMap {
    struct Member {
        uint id;
        string name;
    }
    //bidrectional map
    mapping(address=>Member) memberMap;
    mapping(uint=>address) addressById;
    mapping(string=>address) addressByName;
    function addMember (uint _id, string memory _name) public {
       Member memory m=Member(_id, _name);
       memberMap[msg.sender]=m;
       //saving into a bidiretional map to get address by id
       addressById[_id]=msg.sender;
       //saving into a bidiretional map to get address by name
       addressByName[_name]=msg.sender;
    }
    //get Member by id
    function getMemberById(uint _id) view public returns(uint, string memory) {
        Member memory my = memberMap[addressById[_id]];
        return (my.id, my.name);
    }
    // get address (not Member) by name
    function getMemberAddressByName(string memory _name) public view returns(address) {
        return addressByName[_name];
    }
    function getMember(address addr) view public returns (uint, string memory) {
        Member memory m=memberMap[addr];
        return (m.id, m.name);
    }
}

Writing src/MembersMap.sol


- require 는 송금이 되었는지 확인하는 경우에 많이쓰임 (보통 함수의 맨 앞에 쓰임)
- assert는 내부조건이 모두 충족이 되었는지 확인하는 경우에 많이쓰임 (보통 함수의 맨 뒤에 쓰임)

In [80]:
%%writefile src/ExceptionTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract ExceptionTest {
    address owner;
    constructor() {//constructor() public {
        owner=msg.sender;
    }
    function requireException() view public returns(string memory) {
        //if (msg.sender != owner) { throw; } // not this way to throw an exception
        require(msg.sender != owner, "Sorry! You are owner. Require failed...");
        return "require condition met";
    }
    function assertException() view public returns(string memory) {
        assert(msg.sender == owner);
        return "asserted";  //do this line only 'assert' is succeeded
    }
    function revertException() view public returns(string memory) {
        if(msg.sender != owner) {
            revert("Sorry! You are NOT owner. Reverted...");
            //return "reverted";  // this line can't be reached if revert is executed 
        } else {
            return "not reverted";
        }
    }
    function raiseException() pure public {
        int x=0;
        int y=0;
        x/y;    //divide by zero
    }
}

Writing src/ExceptionTest.sol


# 컨트랙 결합 - 컨트랙이 한 파일안에 존재할 경우 (C2로 배포하여 C1을 참조)

In [1]:
%%writefile src/C1C2.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;
//pragma solidity ^0.6;
//pragma solidity 0.4.21;

contract C1 {
    uint128 v1;
    function set(uint128 _v1) public {
        v1=_v1;
    }
    function get() public view returns(uint128) {
        return v1;
    }
    function get7() public pure returns(uint128) {
        return 7;
    }
}

contract C2 {
    C1 c1;
    //function C2() public {  //0.4.21 constructor
    constructor() { //constructor() public {    //0.6 constructor
        c1=new C1();
    }
    function set(uint128 _v1) public {
        c1.set(_v1);
    }
    function get() public view returns(uint128) {
        return c1.get();
    }
    function get7() public view returns(uint128) {
        return c1.get7();
    }
    function getC1Address() public view returns(address) {
        return address(c1);
    }
}

Writing src/C1C2.sol


In [2]:
!solc-windows.exe src/C1C2.sol --combined-json abi,bin > src/C1C2.json

In [3]:
%%writefile src/C1C2Deploy.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/C1C2.json");
var _json=JSON.parse(_str)
//var _abiArray=JSON.parse(_json.contracts.sHello2.abi);
//var _abiArray=JSON.parse(_json.contracts["src/C1C2.sol:C2"].abi);
var _abiArray=_json.contracts["src/C1C2.sol:C2"].abi;
//var _bin=_json.contracts.sHello2.bin;
var _bin="0x"+_json.contracts["src/C1C2.sol:C2"].bin;

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000})
        .on('transactionHash', function(hash){
            console.log(">>> transactionHash"+hash);
        })
        .on('receipt', function(receipt){
            console.log(">>> RECEPIT hash: " + receipt.transactionHash + "\n>>> address:" + receipt.contractAddress);
        })
        .on('error', function(error, receipt) {
            console.log(">>> ERROR "+error);
        });
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Writing src/C1C2Deploy.js


In [6]:
!node src/C1C2Deploy.js

Deploying the contract from 0x9F4fE137D843278102461B24eD20c6C6C222a423
>>> transactionHash0x4f882a8ddc24c545257806feae4cf248d4d578b2772ebadbd03394df2ad02b2f
>>> RECEPIT hash: 0x4f882a8ddc24c545257806feae4cf248d4d578b2772ebadbd03394df2ad02b2f
>>> address:0xfa85A2051b9a695a19B67AB793aDb8A259132378
---> The contract deployed to: 0xfa85A2051b9a695a19B67AB793aDb8A259132378


In [7]:
%%writefile src/C1C2Use.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/C1C2.json");
var _json=JSON.parse(_str)
//var _abiArray=JSON.parse(_json.contracts.sHello2.abi);
//var _abiArray=JSON.parse(_json.contracts["src/C1C2.sol:C2"].abi);
var _abiArray=_json.contracts["src/C1C2.sol:C2"].abi;

var c2 = new web3.eth.Contract(_abiArray, "0xfa85A2051b9a695a19B67AB793aDb8A259132378");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    c2.methods.get7().call().then(function(res) {console.log("C1 get7(): " + res)});
    await c2.methods.set(9).send({from: accounts[0],gas:50000}, function(err,res) {
        console.log("setting 9..."+res);
    });
    c2.methods.get().call().then(function(res) {console.log("C1 get(): " + res)});
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    //hello.methods.kill().send({from: accounts[0]})
}

doIt()

Writing src/C1C2Use.js


In [8]:
!node src/C1C2Use.js

Account: 0x9F4fE137D843278102461B24eD20c6C6C222a423
Balance before: 999999112166000000000
C1 get7(): 7
setting 9...0x7c5166d65175d4eb478df9fa2238337a5856cfa577dcfa65afa55b33c2fb5a2c
Balance after: 999999013510000000000
Balance diff: 98656000016384
C1 get(): 9


# 컨트랙 결합 - 컨트랙이 다른 파일안에 존재할 경우 (C1을 배포하여 주소를 참조하여 C2에 적용)

In [9]:
%%writefile src/C1.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;
//pragma solidity ^0.6;
//pragma solidity 0.4.21;

contract C1 {
    uint128 v1;
    function set(uint128 _v1) public {
        v1=_v1;
    }
    function get() public view returns(uint128) {
        return v1;
    }
    function get7() public pure returns(uint128) {
        return 7;
    }
}

Writing src/C1.sol


In [10]:
!solc-windows.exe src/C1.sol --combined-json abi,bin > src/C1.json

In [12]:
!type src\C1.json

{"contracts":{"src/C1.sol:C1":{"abi":[{"inputs":[],"name":"get","outputs":[{"internalType":"uint128","name":"","type":"uint128"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"get7","outputs":[{"internalType":"uint128","name":"","type":"uint128"}],"stateMutability":"pure","type":"function"},{"inputs":[{"internalType":"uint128","name":"_v1","type":"uint128"}],"name":"set","outputs":[],"stateMutability":"nonpayable","type":"function"}],"bin":"608060405234801561001057600080fd5b506101d8806100206000396000f3fe608060405234801561001057600080fd5b50600436106100415760003560e01c8063086949b7146100465780634178462f146100645780636d4ce63c14610080575b600080fd5b61004e61009e565b60405161005b9190610154565b60405180910390f35b61007e6004803603810190610079919061011c565b6100a7565b005b6100886100e2565b6040516100959190610154565b60405180910390f35b60006007905090565b806000806101000a8154816fffffffffffffffffffffffffffffffff02191690836fffffffffffffffffffffffffffffffff16021790555050565b600080600090549061

In [13]:
%%writefile src/C1Deploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/C1.json");
var _json = JSON.parse(_str)
//var _abiArray = JSON.parse(_json.contracts.sHello2.abi);
//var _abiArray = JSON.parse(_json.contracts["src/C1.sol:C1"].abi);
var _abiArray = _json.contracts["src/C1.sol:C1"].abi;
//var _bin = _json.contracts.sHello2.bin;
var _bin = "0x"+_json.contracts["src/C1.sol:C1"].bin;

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000})
        .on('transactionHash', function(hash){
            console.log(">>> transactionHash" + hash);
        })
        .on('receipt', function(receipt){
            console.log(">>> RECEPIT hash: " + receipt.transactionHash + "\n>>> address:" + receipt.contractAddress);
        })
        .on('error', function(error, receipt) {
            console.log(">>> ERROR " + error);
        });
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Writing src/C1Deploy.js


In [14]:
!node src/C1Deploy.js

Deploying the contract from 0x9F4fE137D843278102461B24eD20c6C6C222a423
>>> transactionHash0xf6811d2ce3bb036d86bb15cbad8358aeb0ecb09ba26921a84c29819877175d1c
>>> RECEPIT hash: 0xf6811d2ce3bb036d86bb15cbad8358aeb0ecb09ba26921a84c29819877175d1c
>>> address:0x87A25DA685c13140dEF0D5B890e0D4692FEc54a9
---> The contract deployed to: 0x87A25DA685c13140dEF0D5B890e0D4692FEc54a9


In [38]:
%%writefile src/C1Use.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/C1.json");
var _json=JSON.parse(_str)
//var _abiArray=JSON.parse(_json.contracts.sHello2.abi);
//var _abiArray=JSON.parse(_json.contracts["src/C1.sol:C1"].abi);
var _abiArray = _json.contracts["src/C1.sol:C1"].abi;

var c1 = new web3.eth.Contract(_abiArray, "0x87A25DA685c13140dEF0D5B890e0D4692FEc54a9");
async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    c1.methods.get7().call().then(console.log);
    await c1.methods.set(9).send({from: accounts[0],gas:50000});
    c1.methods.get().call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    //hello.methods.kill().send({from: accounts[0]})
}

doIt()

Overwriting src/C1Use.js


In [39]:
!node src/C1Use.js

Account: 0x9F4fE137D843278102461B24eD20c6C6C222a423
Balance before: 999996705618000000000
7
Balance after: 999996657852000000000
Balance diff: 47765999976448
9


In [20]:
%%writefile src/C2.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;
//pragma solidity 0.6;
//pragma solidity 0.4.21;
import "./C1.sol";

contract C2 {
    C1 c1;
    //function C2() public {  //0.4.21 constructor
    constructor() {    //constructor() public {    //0.6 constructor
        c1=new C1();
    }
    function setC1(address _addressOfC1) public {
        c1 = C1(_addressOfC1);
    }
    function set(uint128 _v1) public {
        c1.set(_v1);
    }
    function get() public view returns(uint128) {
        return c1.get();
    }
    function get7() public view returns(uint128) {
        return c1.get7();
    }
    function getC1Address() public view returns(address) {
        return address(c1);
    }
}

Writing src/C2.sol


In [21]:
!solc-windows.exe src/C2.sol --combined-json abi,bin > src/C2.json

In [40]:
!type src\C2.json

{"contracts":{"src/C1.sol:C1":{"abi":[{"inputs":[],"name":"get","outputs":[{"internalType":"uint128","name":"","type":"uint128"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"get7","outputs":[{"internalType":"uint128","name":"","type":"uint128"}],"stateMutability":"pure","type":"function"},{"inputs":[{"internalType":"uint128","name":"_v1","type":"uint128"}],"name":"set","outputs":[],"stateMutability":"nonpayable","type":"function"}],"bin":"608060405234801561001057600080fd5b506101d8806100206000396000f3fe608060405234801561001057600080fd5b50600436106100415760003560e01c8063086949b7146100465780634178462f146100645780636d4ce63c14610080575b600080fd5b61004e61009e565b60405161005b9190610154565b60405180910390f35b61007e6004803603810190610079919061011c565b6100a7565b005b6100886100e2565b6040516100959190610154565b60405180910390f35b60006007905090565b806000806101000a8154816fffffffffffffffffffffffffffffffff02191690836fffffffffffffffffffffffffffffffff16021790555050565b600080600090549061

In [22]:
%%writefile src/C2Deploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/C2.json");
var _json = JSON.parse(_str)
//var _abiArray = JSON.parse(_json.contracts.sHello2.abi);
//var _abiArray = JSON.parse(_json.contracts["src/C2.sol:C2"].abi);
var _abiArray = _json.contracts["src/C1.sol:C1"].abi;
//var _bin = _json.contracts.sHello2.bin;
var _bin = "0x"+_json.contracts["src/C2.sol:C2"].bin;

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000})
        .on('transactionHash', function(hash){
            console.log(">>> transactionHash" + hash);
        })
        .on('receipt', function(receipt){
            console.log(">>> RECEPIT hash: " + receipt.transactionHash + "\n>>> address:" + receipt.contractAddress);
        })
        .on('error', function(error, receipt) {
            console.log(">>> ERROR " + error);
        });
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Writing src/C2Deploy.js


In [23]:
!node src/C2Deploy.js

Deploying the contract from 0x9F4fE137D843278102461B24eD20c6C6C222a423
>>> transactionHash0xe0db72483f4ebb990ed740cf6b25da46e3fe3f5028d6d440c22ace837498e7c2
>>> RECEPIT hash: 0xe0db72483f4ebb990ed740cf6b25da46e3fe3f5028d6d440c22ace837498e7c2
>>> address:0xfF8EB1f456FAeF094ca5a18e58C21f692632A2ec
---> The contract deployed to: 0xfF8EB1f456FAeF094ca5a18e58C21f692632A2ec


In [31]:
%%writefile src/C2Use.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/C2.json");
var _json = JSON.parse(_str)
//var _abiArray = JSON.parse(_json.contracts.sHello2.abi);
//var _abiArray = JSON.parse(_json.contracts["src/C2.sol:C2"].abi);
var _abiArray = _json.contracts["src/C2.sol:C2"].abi;

var c2 = new web3.eth.Contract(_abiArray, "0xfF8EB1f456FAeF094ca5a18e58C21f692632A2ec");
async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    console.log("--- new C1 ---");
    await c2.methods.getC1Address().call(function(err, c1addr) {
        if(!err) console.log("c1 address by 'new': "+c1addr);
    });
    c2.methods.get7().call().then(function(res) { console.log("get7(): "+res) });
    console.log("--- set the above deployed address of C1 ---");
    await c2.methods.setC1("0x87A25DA685c13140dEF0D5B890e0D4692FEc54a9").send({from:accounts[0], gas:50000});
    await c2.methods.getC1Address().call(function(err, c1addr) {
        if(!err) console.log("c1 address by 'setC1()': "+c1addr);
    });
    c2.methods.get7().call().then(console.log);
    await c2.methods.set(222).send({from: accounts[0],gas:50000});
    c2.methods.get().call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    //hello.methods.kill().send({from: accounts[0]})
}

doIt()

Overwriting src/C2Use.js


In [35]:
!node src/C2Use.js

Account: 0x9F4fE137D843278102461B24eD20c6C6C222a423
Balance before: 999996866236000000000
--- new C1 ---
c1 address by 'new': 0x87A25DA685c13140dEF0D5B890e0D4692FEc54a9
--- set the above deployed address of C1 ---
get7(): 7
c1 address by 'setC1()': 0x87A25DA685c13140dEF0D5B890e0D4692FEc54a9
7
Balance after: 999996758984000000000
Balance diff: 107251999965184
222


# 실습문제 - 스퀘어

In [41]:
%%writefile src/SquareArea.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;
//pragma solidity ^0.6.0;
//pragma solidity 0.4.21;

contract Square {
    uint128 length;
    function getLength() public view returns(uint128) {
        return length;
    }
    function setLength(uint128 _length) public {
        length=_length;
    }
    function getDegree() public pure returns(uint128) {
        return 90;
    }
}

contract Area {
    Square s;
    address owner;
    //event PrintLog(uint128);
    //function Area() public {
    constructor() { //constructor() public {
        s = new Square();
        owner = msg.sender;
    }
    function changeSquare(address _addressOfSquare) public {
        s=Square(_addressOfSquare);
    }
    function calcArea() view public returns(uint128) {
        uint128 length = s.getLength();
        uint128 area = length*length;
        //emit PrintLog(area);
        return area;
    }
    function setLength(uint128 _length) public {
        s.setLength(_length);
    }
    function getLength() public view returns(uint128) {
        return s.getLength();
    }
    function getDegree() public view returns(uint128) {
        return s.getDegree();
    }
    function getAddressOfSquare() public view returns(address) {
        return address(s);
    }
}

Writing src/SquareArea.sol


In [42]:
!solc-windows.exe src/SquareArea.sol --combined-json abi,bin > src/SquareArea.json

In [44]:
%%writefile src/SquareAreaDeploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/SquareArea.json");
var _json=JSON.parse(_str)
//var _abiArray = JSON.parse(_json.contracts.sHello2.abi);
//var _abiArray = JSON.parse(_json.contracts["src/SquareArea.sol:Area"].abi);
var _abiArray = _json.contracts["src/SquareArea.sol:Area"].abi;
//var _bin = _json.contracts.sHello2.bin;
var _bin = "0x" + _json.contracts["src/SquareArea.sol:Area"].bin;

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000})
        .on('transactionHash', function(hash){
            console.log(">>> transactionHash"+hash);
        })
        .on('receipt', function(receipt){
            console.log(">>> RECEPIT hash: " + receipt.transactionHash + "\n>>> address:" + receipt.contractAddress);
        })
        .on('error', function(error, receipt) {
            console.log(">>> ERROR "+error);
        });
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy()

Writing src/SquareAreaDeploy.js


In [45]:
!node src/SquareAreaDeploy.js

Deploying the contract from 0x9F4fE137D843278102461B24eD20c6C6C222a423
>>> transactionHash0xc50d349f1a2039b9d28583585ed90d5d4368bdd7543b228d806ef7bc463f6633
>>> RECEPIT hash: 0xc50d349f1a2039b9d28583585ed90d5d4368bdd7543b228d806ef7bc463f6633
>>> address:0xf16d7DAC3FF5136f9d7bbd527D571B3D5Faf3A67
---> The contract deployed to: 0xf16d7DAC3FF5136f9d7bbd527D571B3D5Faf3A67


아래 SquareAreaUse.js 코드 실행시 websocket으로 열면 일부 코드가 실행안될수도있으므로 주의해서 판단할 것

In [52]:
%%writefile src/SquareAreaUse.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));         //nok
//var web3 = new Web3(new Web3.providers.WebsocketProvider("http://117.16.44.45:8345"));  //ok
//var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/SquareArea.json");
var _json = JSON.parse(_str)
//var _abiArray = JSON.parse(_json.contracts.sHello2.abi);
//var _abiArray = JSON.parse(_json.contracts["src/SquareArea.sol:Area"].abi);
var _abiArray = _json.contracts["src/SquareArea.sol:Area"].abi;

async function doIt() {
    var area = new web3.eth.Contract(_abiArray, "0xf16d7DAC3FF5136f9d7bbd527D571B3D5Faf3A67");
    var speed;
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    area.methods.getDegree().call().then(console.log);
    await area.methods.setLength(9).send({from: accounts[0]});
    area.methods.getLength().call().then(console.log);
    //area.methods.calcArea().send({from: accounts[0]});
    area.methods.calcArea().call().then(console.log);
    area.methods.getAddressOfSquare().call().then(function(address) {
        console.log("Square Address: " + address);
    });
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    //process.exit(1); //force exit -> may terminate some functions (speedDownBy10, getSpeed)  
}

doIt()

Overwriting src/SquareAreaUse.js


In [53]:
!node src/SquareAreaUse.js

Account: 0x9F4fE137D843278102461B24eD20c6C6C222a423
Balance before: 999995215940000000000
90
Balance after: 999995157040000000000
Balance diff: 58900000014336
Square Address: 0x1087169f3905F6F0B46fCD0eD79995dE2657b56b
9
81


# 디파이 개발

In [13]:
%%writefile src/MyToken.sol
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

import "@openzeppelin/contracts/token/ERC20/ERC20.sol";

contract MyToken is ERC20 {
    constructor() ERC20("MyToken", "JSL"){
        _mint(msg.sender, 1000000000000000000000000);
        //_mint(msg.sender, 1000000*decimals());
    }
}

Overwriting src/MyToken.sol


In [38]:
!solc-windows.exe @openzeppelin/="C:\\Users\\Administrator\\Code\\201810825\\node_modules\\@openzeppelin\\" --optimize --combined-json abi,bin src/MyToken.sol > src/MyToken.json

In [19]:
!type src\MyToken.json

{"contracts":{"C://Users//Administrator//Code//201810825//node_modules//@openzeppelin/contracts/token/ERC20/ERC20.sol:ERC20":{"abi":[{"inputs":[{"internalType":"string","name":"name_","type":"string"},{"internalType":"string","name":"symbol_","type":"string"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"address","name":"spender","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","

일반 컴파일로는 @뒤에 붙는것들이 인식이 되지 않아서 경로 재지정을 통해 오류를 해결

In [17]:
%%writefile src/MyTokenTest.js
var _abiJson = require('./MyToken.json');
contractName=Object.keys(_abiJson.contracts); 
console.log("- contract name: ", contractName[4]); //reading src/MyToken.sol:MyToken

Writing src/MyTokenTest.js


In [18]:
!node src/MyTokenTest.js

- contract name:  src/MyToken.sol:MyToken


In [20]:
%%writefile src/MyTokenDeployAbiBinFromFile.js
var Web3=require('web3');
var _abiBinJson = require('./MyToken.json');

var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}

contractName=Object.keys(_abiBinJson.contracts);
console.log("- contract name: ", contractName[4]); //or console.log(contractName);

//_abi=_abiBinJson.contracts[contractName[4]].abi
//_bin=_abiBinJson.contracts[contractName[4]].bin


//contractName=Object.keys(_abiJson.contracts); // reading ['src//Timer.sol:Timer'];
//console.log("- contract name: ", contractName[4]); //or console.log(contractName);
_abiArray=JSON.parse(JSON.stringify(_abiBinJson.contracts[contractName[4]].abi));    //JSON parsing needed!!
//_abiArray=JSON.stringify(_abi)
_bin="0x"+_abiBinJson.contracts[contractName[4]].bin;
//console.log("- ABI: " + JSON.stringify(_abiArray));
//console.log("- Bytecode: " + _bin);

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Writing src/MyTokenDeployAbiBinFromFile.js


In [21]:
!node src/MyTokenDeployAbiBinFromFile.js

- contract name:  src/MyToken.sol:MyToken
Deploying the contract from 0xA7c5b2123CDf8cC2f01cE15204D79Ebb8aB226c0
hash: 0x5f64b6f4af1efecf97a8d23622e45ab0c6f5e85095890f4921033ee19e460eee
---> The contract deployed to: 0xba54B9305bbBe728db815ea6cF46A8C5d1Df70bD


In [22]:
%%writefile src/MyTokenUse.js
var Web3=require('web3');
var _abiBinJson = require('./MyToken.json');

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts); 
console.log("- contract name: ", contractName[4]);// reading ['src//Timer.sol:Timer']
var _abiArray=JSON.parse(JSON.stringify(_abiBinJson.contracts[contractName[4]].abi));    //JSON parsing needed!!

var _test = new web3.eth.Contract(_abiArray, '0xba54B9305bbBe728db815ea6cF46A8C5d1Df70bD');

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    
    const name = await _test.methods.name().call();
    console.log("name: " + name);
    const decimals = await _test.methods.decimals().call();
    console.log("decimals: " + decimals);
    const value = await _test.methods.totalSupply().call();
    console.log("totalSupply: " + value);

    console.log("-----1 before transfer-----");
    console.log("1-1 balanceOfAccount0: " + await _test.methods.balanceOf(accounts[0]).call());
    console.log("1-2 balanceOfAccount1: " + await _test.methods.balanceOf(accounts[1]).call());
    await _test.methods.transfer(accounts[1], 1)
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
    console.log("-----2 after transfer-----");
    console.log("2-1 balanceOfAccount0: " + await _test.methods.balanceOf(accounts[0]).call());
    console.log("2-2 balanceOfAccount1: " + await _test.methods.balanceOf(accounts[1]).call());
    console.log("-----3 after approve and transferFrom-----");    
    var addrTo = accounts[0];
    console.log("3-1 allowanceOfAddrTo: "+ await _test.methods.allowance(accounts[0], addrTo).call());
    await _test.methods.approve(addrTo, 111)
        .send({from: accounts[0], gas: 100000});
    console.log("3-2 allowanceOfAddrTo 111: "+ await _test.methods.allowance(accounts[0], addrTo).call());
    await _test.methods.transferFrom(addrTo, accounts[1], 1)
        .send({from: accounts[0], gas: 3641240, gasPrice: '1000000000'})
    console.log("3-3 balanceOfAccount0: " + await _test.methods.balanceOf(accounts[0]).call());
    console.log("3-4 balanceOfAccount1: " + await _test.methods.balanceOf(accounts[1]).call());
    //const value = await _test.methods.myFunction()
    //    .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));});
    //console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}
doIt()

Writing src/MyTokenUse.js


In [23]:
!node src/MyTokenUse.js

- contract name:  src/MyToken.sol:MyToken
Account: 0xA7c5b2123CDf8cC2f01cE15204D79Ebb8aB226c0
Balance before: 999998668672000000000
name: MyToken
decimals: 18
totalSupply: 1000000000000000000000000
-----1 before transfer-----
1-1 balanceOfAccount0: 1000000000000000000000000
1-2 balanceOfAccount1: 0
-----2 after transfer-----
2-1 balanceOfAccount0: 999999999999999999999999
2-2 balanceOfAccount1: 1
-----3 after approve and transferFrom-----
3-1 allowanceOfAddrTo: 0
3-2 allowanceOfAddrTo 111: 111
3-3 balanceOfAccount0: 999999999999999999999998
3-4 balanceOfAccount1: 2
Balance after: 999998481953000000000
Balance diff: 186719000068096


QmXfKWE57ZCb45AZPJnU2rburapxQuDka5ZADa4VDq3sWH

In [65]:
%%writefile src/nftHelloUri.json
{
    "name": "Hello DApp Screens",
    "description": "Screens to Deploy the Hello DApp",
    "image": "https://ipfs.io/ipfs/QmXfKWE57ZCb45AZPJnU2rburapxQuDka5ZADa4VDq3sWH"
}

Overwriting src/nftHelloUri.json


In [66]:
from pinatapy import PinataPy
APIKey="dd919f2451a5e3430fde"
APISecret="7d30ee7c789e18ca477c89cd69719c597382673248c3e0f26d1f24a58469914c"
pinata = PinataPy(APIKey, APISecret)

In [67]:
# Upload the file
result = pinata.pin_file_to_ipfs("src/nftHelloUri.json")
# Should return the CID (unique identifier) of the file
print(result)

{'IpfsHash': 'QmTUFcjfuTJdjME1WYvY18srF9Rj2VDSwHUVW5Uqw72Q7i', 'PinSize': 192, 'Timestamp': '2022-06-13T08:07:06.930Z', 'isDuplicate': True}


API Key: dd919f2451a5e3430fde
 API Secret: 7d30ee7c789e18ca477c89cd69719c597382673248c3e0f26d1f24a58469914c
 JWT: eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ1c2VySW5mb3JtYXRpb24iOnsiaWQiOiJjNzQ4MjhkYS1kODgxLTQwYzctYTRjZS0xODA2NmQ1Mjg4NDAiLCJlbWFpbCI6Imhvbmdjb21hQG5hdmVyLmNvbSIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJwaW5fcG9saWN5Ijp7InJlZ2lvbnMiOlt7ImlkIjoiRlJBMSIsImRlc2lyZWRSZXBsaWNhdGlvbkNvdW50IjoxfSx7ImlkIjoiTllDMSIsImRlc2lyZWRSZXBsaWNhdGlvbkNvdW50IjoxfV0sInZlcnNpb24iOjF9LCJtZmFfZW5hYmxlZCI6ZmFsc2UsInN0YXR1cyI6IkFDVElWRSJ9LCJhdXRoZW50aWNhdGlvblR5cGUiOiJzY29wZWRLZXkiLCJzY29wZWRLZXlLZXkiOiJkZDkxOWYyNDUxYTVlMzQzMGZkZSIsInNjb3BlZEtleVNlY3JldCI6IjdkMzBlZTdjNzg5ZTE4Y2E0NzdjODljZDY5NzE5YzU5NzM4MjY3MzI0OGMzZTBmMjZkMWYyNGE1ODQ2OTkxNGMiLCJpYXQiOjE2NTUxMDc2MDB9.LLsqN6ZGULGN7BNKn5cIFeiUNwDKD-YrZgiygTY4hhA

# NFT 만들기

In [73]:
%%writefile src/MyNFT.sol
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

import "@openzeppelin/contracts/token/ERC721/ERC721.sol";
//import "https://github.com/OpenZeppelin/openzeppelin-contracts/blob/v4.4.0/contracts/token/ERC721/ERC721.sol";
import "@openzeppelin/contracts/utils/Counters.sol";
//import "https://github.com/OpenZeppelin/openzeppelin-contracts/blob/v4.4.0/contracts/utils/Counters.sol";

contract MyNFT is ERC721 {
    using Counters for Counters.Counter;
    Counters.Counter private _tokenIds;//NFT id
    
    constructor() ERC721("Hello DApp Screens", "JSL") {}
    // override
    // does not save toeknURI in the state var
    function _baseURI() internal pure override returns (string memory) {
        return "https://ipfs.io/ipfs/QmbdwcmzYacLZHLBAhAsaJm2aPV1bzBjFjweD1UgZzkEtA";
    }

    function mint(address to) public returns(uint256) {
        //require(_tokenIds.current() < 3); //only 3 to mint
        _tokenIds.increment(); // add 1 by minting
        uint256 newTokenId = _tokenIds.current();
        _mint(to, newTokenId); // or _safeMint(to, newTokenId);

        return newTokenId;
    }
}

Overwriting src/MyNFT.sol


In [1]:
!solc-windows.exe @openzeppelin/="C:\\Users\\Administrator\\Code\\201810825\\node_modules\\@openzeppelin\\" --optimize --combined-json abi,bin src/MyNFT.sol > src/MyNFT.json

Error: Source file requires different compiler version (current compiler is 0.8.0+commit.c7dfd78e.Windows.msvc) - note that nightly builds are considered to be strictly less than the released version
 --> C://Users//Administrator//Code//201810825//node_modules//@openzeppelin/contracts/utils/Address.sol:4:1:
  |
4 | pragma solidity ^0.8.1;
  | ^^^^^^^^^^^^^^^^^^^^^^^

